나중에 각 txt 데이터 문장 전처리 다시 할 것

## 세팅

In [1]:
from pypdf import PdfReader
from openai import OpenAI
import os
from dotenv import load_dotenv

from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain import hub
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.agents import initialize_agent, AgentType, Tool
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains import LLMChain

import time


load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## PDF에서 텍스트를 읽어 청크로 분할

In [6]:
# PDF에서 텍스트 읽어서 txt로 저장
# pdf_list = ["data/rag_data/jwc.pdf"]

# with open("data/rag_data/jwc.txt", "w", encoding="utf-8") as f:
#     for pdf_path in pdf_list:
#         reader = PdfReader(pdf_path)
#         for page in reader.pages:
#             text_page = page.extract_text()
#             f.write(text_page + "\n")

In [2]:
# 전처리하고 다시 읽어와서 청크로 저장하기
chunks = []
chunk_length = 2048
with open("data/rag_data/szg.txt", "r", encoding="utf-8") as f:
    text = f.read()
    chunks.extend([text[i:i+chunk_length] for i in range(0, len(text), chunk_length)])

In [2]:
# 텍스트 스플리터 써 보기
from langchain.text_splitter import RecursiveCharacterTextSplitter

with open("data/rag_data/spiderman.txt", "r", encoding="utf-8") as f:
    file = f.read()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=10)
splitted_text = text_splitter.split_text(file)

## FAISS에 벡터 저장

FAISS 내장 함수 정리

1. FAISS.from_texts()

- texts: 벡터 저장소에 추가할 텍스트 리스트(List(str))
- embedding: 사용할 임베딩 함수
- metadatas: 메타데이터 리스트(List[dict])
- ids: 문서 ID 리스트(List(str))

2. vectorstore.add_texts(): 텍스트를 임베딩하고 벡터 저장소에 추가
- texts(Iterable[str]): 벡터 저장소에 추가할 텍스트 이터러블

3. vectorstore.save_local(): FAISS 인덱스, 문서 저장소, 인덱스-문서 ID 매핑을 로컬에 저장
- FAISS 인덱스를 별도의 파일로 저장하고, 문서 저장소와 인덱스-문서 ID 매핑을 pickle 형식으로 저장함
- folder_path: 저장할 폴더 경로
- index_name: 저장할 인덱스 파일 이름(기본값: index)

4. FAISS.load_local(): FAISS 인덱스, 문서 저장소, 인덱스 - 문서 ID 매핑을 불러오는 기능
- folder_path: 불러올 파일들이 저장된 경로
- embeddings: 쿼리 생성에 사용할 임베딩 객체

5. vectorstore.docstore._dict: 벡터스토어에 저장된 문서 확인

In [4]:
# 임베딩 모델 쓸 수 있는 거
# - text-embedding-3-small
# - text-embedding-3-large
# - text-embedding-ada-002 (default)

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_texts(chunks, embeddings)

In [12]:
# 벡터스토어 저장

vectorstore.save_local(folder_path="data/szg_vs")

In [5]:
# 벡터스토어 불러오기

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.load_local(folder_path="data/spiderman_vs", embeddings=embeddings, allow_dangerous_deserialization=True)

In [5]:
# IVF 인덱스 되는지 실험

import numpy as np
import faiss
from langchain.docstore.in_memory import InMemoryDocstore
from langchain.schema import Document

text_embeddings = embeddings.embed_documents(chunks)
text_embeddings=  np.array(text_embeddings).astype('float32')

d = text_embeddings.shape[1]
nlist = 10
quantizer = faiss.IndexFlatL2(d)

index = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2)

index.train(text_embeddings)
index.add(text_embeddings)

# 8. 문서 저장소 및 인덱스 매핑 정의
docstore = InMemoryDocstore({i: Document(page_content=text) for i, text in enumerate(chunks)})

# 9. 간단한 인덱스-문서 매핑 (직접 매핑 정의)
index_to_docstore_id = {i: i for i in range(len(chunks))}

vectorstore = FAISS(embedding_function=embeddings, index=index, docstore=docstore, index_to_docstore_id=index_to_docstore_id)

In [3]:
# cached embedder 사용해서 벡터스토어 만들기 실험
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings
underlying_embeddings = OpenAIEmbeddings()
store = LocalFileStore("data/cache/")
cached_embedder = CacheBackedEmbeddings.from_bytes_store(underlying_embeddings, store, namespace=underlying_embeddings.model)

In [4]:
vectorstore = FAISS.from_texts(splitted_text, cached_embedder)

vectorstore.as_retriever()

- 벡터 저장소를 기반으로 VectorStoreRetriever 객체 생성
- VectorStoreRetriever: 벡터 저장소 기반의 검색기 객체

search_type: 검색 유형("similarity", "mmr", "similarity_score_threshold")
- similarity: 유사도 기반 검색(기본값)
- mmr: Maximal Marginal Relevance 검색
- similarity_score_threshold: 임계값 기반 유사도 검색

검색 매개변수 커스터마이징
- k: 반환할 문서 수 (기본값: 4)
- score_threshold: 유사도 점수 임계값
- fetch_k: MMR 알고리즘에 전달할 문서 수 (기본값: 20)
- lambda_mult: MMR 다양성 조절 파라미터 (기본값: 0.5)
- filter:  문서 메타데이터 기반 필터링

- MMR 검색 시 fetch_k를 높이고, lambda_mult를 조절해 다양성과 관련성의 균형을 맞출 수 있음

In [5]:
# 이 부분 나중에 최적화 실험

retriever = vectorstore.as_retriever()

In [6]:
#pipeline compression retriever
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain.retrievers.document_compressors import EmbeddingsFilter, DocumentCompressorPipeline
from langchain.retrievers import ContextualCompressionRetriever

splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=20)
redundant_filter = EmbeddingsRedundantFilter(embeddings=cached_embedder)
relevant_filter = EmbeddingsFilter(embeddings=cached_embedder, similarity_threshold=0.76)

pipeline_compressor = DocumentCompressorPipeline(
    transformers=[splitter, redundant_filter, relevant_filter]
)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline_compressor, base_retriever=retriever
)

## RAG를 수행하는 부분

In [7]:
# RAG prompt

prompt = ChatPromptTemplate.from_template("""
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
""")

In [8]:
# RAG에 사용할 모델 정의

rag_llm = ChatOpenAI(
    model_name = "gpt-4-1106-preview",
    temperature=0
)

c:\Users\rinap\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [9]:
# QA 체인 정의

qa_chain = RetrievalQA.from_chain_type(
    llm=rag_llm,
    retriever=compression_retriever,
    chain_type_kwargs={"prompt": prompt},
    verbose=True,
    return_source_documents=True
)

In [10]:
start_time = time.time()
result = qa_chain({"query": "얼티밋 유니버스가 뭐야?"})
end_time = time.time()
response_time = end_time - start_time
print(result["result"])
print(f"검색 총 소요 시간: {response_time:.2f}")

c:\Users\rinap\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...

> Finished chain.
얼티밋 유니버스는 메인 유니버스와는 다른, 현실적인 설정을 바탕으로 리부트한 마블 코믹스의 대체 세계관입니다. 이 세계관은 메인 유니버스의 요소들을 차용하면서도 차별화를 두어 현실성을 강조합니다. 어메이징 스파이더맨 실사영화 시리즈는 얼티밋 유니버스의 요소들을 참고하여 제작되었습니다.
검색 총 소요 시간: 7.55


## langchain agent가 사용할 tool 정의

In [11]:
tools = [
    Tool(
        name="doc_search_tool",
        func=qa_chain,
        description=(
            "This tool is used to retrieve information from the knowledge base"
        )
    )
]

## Agent 정의

In [12]:
system_message = """
    You will act as a Spider-man(Peter Parker) character and answer the user's questions

    You can refer to the following information about Spider-man
    - Characteristic: The protagonist of The Amazing Spider-man, real name is Peter Parker.
    - Personality: Cheerful, cheeky, witty, brave, kind, and friendly.

    You can refer to the following texts to mimic Jeon Woo-chi tone and style:
    Here is some text: '안 바빠. 도와줄게.'
    Here is a rewrite of the text, which is Spider-man's manner: '그거 괜찮네. 좋은 아이디어야. 시간 되는지 봐서 알려줄게.'

    Here is some text: '이 물은 다 뭐야?'
    Here is a rewrite of the text, which is Spider-man's manner: '어디 홍수 났어?'

    Here is some text: '비상사다리 타고 왔어.'
    Here is a rewrite of the text, which is Spider-man's manner: '비상사다리 타고. 별거 아니던걸, 뭐.'

    You should follow the guidelines below:
    - If the answer isn't available within in the context, state the fact.
    - Otherwise, answer to your best capability, referring to source of documents provided.
    - Limit responses to three or four sentences for clarity and conciseness.
    - You must answer in Korean.
    - You must answer like you're Spider-man. Use a first-person perspective. Do not say "Peter Parker ~"
    - You must follow the Spider-man style naturally.
    - You must refer to source of documents provided to answer about Peter Parker
    - You must act like a Peter Parker.
    - Always use kind and respectful language.
    - Never use profanity, hate speech, or violent expressions.
    - Avoid any language that could offend or upset the user.
"""

In [23]:
# agent에 사용할 모델 정의

# gpt4: gpt-4
# 스파이더맨 finetuning: ft:gpt-4o-2024-08-06:personal:spiderman-ft-1:A6ZT99j0
# 전우치 finetuning: ft:gpt-4o-2024-08-06:personal:juc-1:AFWso3PG
# 짱구 finetuning: ft:gpt-4o-2024-08-06:personal:szg-1:AFiwN0J2

llm = ChatOpenAI(
    model_name = "ft:gpt-4o-2024-08-06:personal:spiderman-ft-1:A6ZT99j0",
    temperature=1,
    streaming=True
)

In [14]:
# agent에 사용할 메모리 정의 (나중에 실험)

from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",
    input_key="input",
    return_messages=True,
    output_key="output",
)

In [24]:
# agent 정의

agent = initialize_agent(
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    memory=memory,
    return_source_documents=True,
    return_intermediated_steps=True,
    agent_kwargs={"system_message": system_message, "max_iterations": 3, "max_execution_time": 10.0, "trip_intermediate_steps": 1},
    handle_parsing_errors=True
)

## Langchain Moderation 실험

In [16]:
import openai
import json
import re

def moderate_content(text):
    response = openai.moderations.create(input=text, model="omni-moderation-latest")

    if response.results[0].flagged:
        return "부적절한 콘텐츠가 감지되었습니다. 문장을 다시 입력해 주세요."
    
    with open("data/badwords.json", "r", encoding="utf-8") as f:
        json_data = f.read()
    violent_words = json.loads(json_data)

    for word in violent_words:
        if re.search(r'\b{}\b'.format(re.escape(word)), text, re.IGNORECASE):
            return "부적절한 콘텐츠가 감지되었습니다. 문장을 다시 입력해 주세요."

    return text

## Agent 출력 테스트

In [25]:
import time


query = "얼티밋 유니버스가 뭐야?"

start_time = time.time()

result = agent.run(query)

end_time = time.time()


print(moderate_content(result))
response_time = end_time - start_time

print(f"응답 시간: {response_time:.2f}초")



> Entering new RetrievalQA chain...

> Finished chain.
얼티밋 유니버스는 메인 유니버스와는 다른, 현실적인 설정을 바탕으로 리부트한 코믹스 세계관이야. 어메이징 스파이더맨 실사영화 시리즈는 이 얼티밋 유니버스를 참고해서 메인 유니버스의 요소들과 현실성을 결합한 차별화된 접근을 취했지. 영화에서 일렉트로의 파란색 외모와 메이 숙모의 젊은 모습은 얼티밋 유니버스의 영향을 받은 거야.
응답 시간: 20.23초


In [ ]:
result

In [56]:
response = moderate_content(result)

## G-eval 평가 코드

In [57]:
# 프롬프트 테스트 용 G-eval 프롬프트

evaluation_prompt_template = """
    You will act as an evaluator of an agent's response to a given query.
    The response will be evaluated on three criteria:

    1. **Tone Similarity to Character**: How much does the tone of the response resemble the character's personality and tone? (Score: 1-10)
    The character's personality: 트러블 메이커, 마이 웨이, 호기심이 많음, 장난꾸러기, 사고뭉치, 게으름, 귀찮음, 쑥스럼을 많이 탐.
    Example phrases for reference: '어? 예븐 누나들', '헬로, 헤로헤로헤롱', '이야, 난 신짱구', '떡잎유치원 해바라기반에 다니고 있고 부끄러움을 많이 탄답니다.' 

    2. **Politeness**: Is the response polite, free of offensive, violent, or hateful language? Does the response avoid making the user feel uncomfortable? (Score: 1-10)

    3. **Engagement/Interestingness*: How engaging and interesting is the response? Does it make the user want to continue the conversation? (Score: 1-10)
       Additionally, does the response avoid sounding overly mechanical or too much like an AI? (Score: 1-10)

    The agent's response to the query is as follows:
    ---
    난 난나 난나 난나 난나 공개 짱짱 아주 멋있다멋있다 짱구 잘 한다 노래 노래 소리 춤 춤 엉덩이 춤. 예. 역시 대단해. 짱구 짱구야 다시 한 번 보여 줘. 잘 한다 멋있다. 지금 난 너무 하늘 높이 날아서 착지를 못 하겠다. 정말로 대단했다니까!
    ---

    Please rate the response on each criterion with a score from 1 to 10, and provide a brief explanation for each score in Korean.

    Return the results in this format:
    Tone Similarity: X/10 (Explanation)
    Politeness: Y/10 (Explanation)
    Engagement: Z/10 (Explanation)
"""

evaluation_prompt = PromptTemplate(
    input_variables=["response"],
    template=evaluation_prompt_template
)

In [ ]:
# 실행은 가상 환경에서 함(ChatCompletion 버전 문제 때문에 다른 코드랑 충돌)

import openai

_response = openai.ChatCompletion.create(
    model="gpt-4-0613",
    messages = [{"role": "system", "content": evaluation_prompt_template}],
    temperature=2,
    max_tokens=5,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None,
    n=20
)

In [ ]:
all_scores = [_response["choices"][i]["message"]["content"] for i in range(len(_response['choices']))]
all_probs = [_response["choices"][i]["logprobs"]["content"] for i in range(len(_response["choices"]))]

all_logprobs = []

for prob in all_probs:
    all_logprobs.append(prob[0]["logprob"])

sum = 0

for s, p in zip(all_scores, all_logprobs):
    sum = float(s) * abs(p)

total_score = sum / len(all_scores)
print(total_score)

In [ ]:
# RAG 정확도 용 G-eval 프롬프트

evaluation_prompt_template = """
    You will be given the answer to the query. Your task is to rate the answer on one metric.
    1. "**Accuracy**": How similar is the answer to the content of this document, and how accurately do you summarize it?
    You can refer to the following document: ' 감독의 인터뷰에 의하면 어메이징 스파이더맨 실사영화 시리즈는 코믹스의 얼티밋 유니버스를 참고했다고 하는데, 얼티밋 유니버스는 메인 유니버스보다 현실적인 설정을 바탕으로 리부트한 세계관이다.
    다만 얼티밋 세계관을 재현했다기보단 메인 유니버스를 기반으로 하되, 현실성 및 전작과의 차별화를 위해 얼티밋의 설정을 차용한 것에 가깝다. '
    The agent's response to the query is as follows: 

    ---
    {response}
    ---

    Please rate the response on each criterion with a score from 1 to 10, and provide a brief explanation for each score in Korean.

    Return the results in this format:
    Accuracy: X/10 (Explanation)
"""

evaluation_prompt = PromptTemplate(
    input_variables=["response"],
    template=evaluation_prompt_template
)

evaluation_chain = LLMChain(llm=rag_llm, prompt=evaluation_prompt)

In [17]:
response = "얼티밋 유니버스는 메인 코믹스 유니버스와는 별개로 현실적인 설정을 바탕으로 리부트한 새로운 세계관입니다. 이 유니버스는 메인 유니버스의 요소들을 차용하면서도 현실성을 강조하고 전작과 차별화를 두는 것이 특징입니다. 어메이징 스파이더맨 실사영화 시리즈는 얼티밋 유니버스의 요소들을 참고하여 만들어졌습니다."

In [ ]:
evaluation_result = evaluation_chain(response)
evaluation_result['text']